# Recruit Restaurant Visitor Forecasting

## Prerequisites
Please make sure the following Python distributions and packages were installed.

* [Anaconda](https://anaconda.org)
* [XGBoost](https://github.com/dmlc/xgboost)
* [LightGBM](https://github.com/Microsoft/LightGBM) - not needed by week 1
* [Keras](https://keras.io) - not needed by week 1
* [Tensorflow](https://www.tensorflow.org) - not needed by week 1
* [Bayesian Optimization](https://github.com/fmfn/BayesianOptimization) - not needed by week 1
* [seaborn](https://seaborn.pydata.org)
* [bokeh](http://bokeh.pydata.org)

You'll also need to create the following sub-folders in your working folder:

* input
   
   To store all the data files downloaded from Kaggle
   
   
* output
    
    To store submission files
   
   
* python
    
    To store python scripts and ipython notebooks including this one.

In [1]:
import numpy as np
from scipy import sparse
import pandas as pd
import xgboost as xgb
import re
import string
import time
import seaborn as sns
import itertools
# import lightgbm as lgb - will be used in following weeks
# from bayes_opt import BayesianOptimization - will be used in following weeks
import seaborn as sns
import matplotlib.pyplot as plt

import random
from sklearn import preprocessing, pipeline, metrics, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import Imputer
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.neural_network import MLPClassifier
from IPython.display import display
import datetime

%matplotlib inline 


/Users/jyguo/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Data initialization

In [2]:
train_data = pd.read_csv('../input/air_visit_data.csv')
air_store_info = pd.read_csv('../input/air_store_info.csv')
hpg_store_info = pd.read_csv('../input/hpg_store_info.csv')
air_reserve = pd.read_csv('../input/air_reserve.csv')
hpg_reserve = pd.read_csv('../input/hpg_reserve.csv')
store_id_relation = pd.read_csv('../input/store_id_relation.csv')
test_data = pd.read_csv('../input/sample_submission.csv')
date_info = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
train_size = train_data.shape[0]

# Data preprocessing

### Basic preprocessing
#### Take a look at train and test data

In [3]:
display(train_data.head())
display(test_data.head())

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


#### Split air_store_id and visit_date from id for test data

In [4]:
test_data['visit_date'] = test_data['id'].map(lambda x: str(x).split('_')[2])
test_data['air_store_id'] = test_data['id'].map(lambda x: '_'.join(x.split('_')[:2]))
test_data.head()

,id,visitors,visit_date,air_store_id
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9


#### Merge training and testing data
This is to simplify the efforts for feature engineering otherwise we'll have to perform the same transfromations for both train and test.

In [5]:
full_data = pd.concat([train_data,test_data])
display(full_data.head())
display(full_data.tail())

,air_store_id,id,visit_date,visitors
0,air_ba937bf13d40fb24,NaN,2016-01-13,25
1,air_ba937bf13d40fb24,NaN,2016-01-14,32
2,air_ba937bf13d40fb24,NaN,2016-01-15,29
3,air_ba937bf13d40fb24,NaN,2016-01-16,22
4,air_ba937bf13d40fb24,NaN,2016-01-18,6


,air_store_id,id,visit_date,visitors
32014,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-27,2017-05-27,0
32015,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-28,2017-05-28,0
32016,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-29,2017-05-29,0
32017,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-30,2017-05-30,0
32018,air_fff68b929994bfbd,air_fff68b929994bfbd_2017-05-31,2017-05-31,0


## Datetime features

In [6]:
full_data['visit_date'] = pd.to_datetime(full_data['visit_date'])
full_data['dow'] = full_data['visit_date'].dt.dayofweek
full_data['year'] = full_data['visit_date'].dt.year
full_data['month'] = full_data['visit_date'].dt.month
full_data['doy'] = full_data['visit_date'].dt.dayofyear
full_data['dom'] = full_data['visit_date'].dt.days_in_month
full_data['woy'] = full_data['visit_date'].dt.weekofyear
full_data['is_month_end'] = full_data['visit_date'].dt.is_month_end
full_data['visit_date'] = full_data['visit_date'].dt.date
full_data['date_int'] = full_data['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)

### Store information

There are two types of store information: air and hpg which are two websites where customers can make reservations.

#### Split area names

It appears that the column area_name actually contains 3 levels of geographic area
e.g Tōkyō-to Setagaya-ku Taishidō - > 

Tōkyō-to

    Setagaya-ku

        Taishidō
Let's split it into 3 new columns

In [7]:
air_store_info['air_area_lv1'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[0])
air_store_info['air_area_lv2'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[1])
air_store_info['air_area_lv3'] = air_store_info.air_area_name.apply(lambda x:x.split(' ')[2])

hpg_store_info['hpg_area_lv1'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[0])
hpg_store_info['hpg_area_lv2'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[1])
hpg_store_info['hpg_area_lv3'] = hpg_store_info.hpg_area_name.apply(lambda x:x.split(' ')[2])

In [8]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_area_lv1,air_area_lv2,air_area_lv3
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen


#### Create features that based on different levels of area

In [9]:
air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['latitude','longitude']).air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_on_same_addr'}),
                             how='left',
                             on=['latitude','longitude'])


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).air_store_id.count().\
                                    reset_index().rename(columns={'air_store_id':'air_stores_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_air_lv1'}),
                             how='left',
                             on='air_area_lv1')


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby('air_area_lv1').longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_air_lv1'}),
                             how='left',
                             on='air_area_lv1')

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])

air_store_info = pd.merge(air_store_info,
                             air_store_info.groupby(['air_area_lv1','air_area_lv2']).longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_air_lv2'}),
                             how='left',
                             on=['air_area_lv1','air_area_lv2'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['latitude','longitude']).hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_on_same_addr'}),
                             how='left',
                             on=['latitude','longitude'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).hpg_store_id.count().\
                                    reset_index().rename(columns={'hpg_store_id':'hpg_stores_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby('hpg_area_lv1').longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_hpg_lv1'}),
                             how='left',
                             on='hpg_area_lv1')

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.mean().\
                                    reset_index().rename(columns={'latitude':'mean_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.max().\
                                    reset_index().rename(columns={'latitude':'max_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).latitude.min().\
                                    reset_index().rename(columns={'latitude':'min_lat_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])


hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.mean().\
                                    reset_index().rename(columns={'longitude':'mean_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.max().\
                                    reset_index().rename(columns={'longitude':'max_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

hpg_store_info = pd.merge(hpg_store_info,
                             hpg_store_info.groupby(['hpg_area_lv1','hpg_area_lv2']).longitude.min().\
                                    reset_index().rename(columns={'longitude':'min_lon_hpg_lv2'}),
                             how='left',
                             on=['hpg_area_lv1','hpg_area_lv2'])

#### Merge store information for stores that exist in both air and hpg

In [10]:
air_store_info = pd.merge(air_store_info, store_id_relation, how='left', on='air_store_id')
air_store_info = pd.merge(air_store_info, hpg_store_info, how='left', on='hpg_store_id')
air_store_info = air_store_info.rename(columns={'latitude_x':'latitude_air',
                             'longitude_x':'longitude_air',
                             'latitude_y':'latitude_hpg',
                             'longitude_y':'longitude_hpg'})

display(air_store_info.head())

,air_store_id,air_genre_name,air_area_name,latitude_air,longitude_air,air_area_lv1,air_area_lv2,air_area_lv3,air_stores_on_same_addr,air_stores_lv1,...,min_lat_hpg_lv1,mean_lon_hpg_lv1,max_lon_hpg_lv1,min_lon_hpg_lv1,mean_lat_hpg_lv2,max_lat_hpg_lv2,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,Hyōgo-ken,Kōbe-shi,Kumoidōri,17,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō-to,Minato-ku,Shibakōen,51,444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Add store information to full data

In [11]:
full_data = pd.merge(full_data, air_store_info, how='left', on='air_store_id')
display(full_data.head())

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,min_lat_hpg_lv1,mean_lon_hpg_lv1,max_lon_hpg_lv1,min_lon_hpg_lv1,mean_lat_hpg_lv2,max_lat_hpg_lv2,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reservation information

In [12]:
print ('before')
display(air_reserve.head())
display(hpg_reserve.head())

air_reserve['visit_date'] = air_reserve['visit_datetime'].apply(lambda x:x[:10])
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].apply(lambda x:x[:10])
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].dt.date
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['visit_date'] = air_reserve['visit_datetime'].dt.date
air_reserve['reserve_datetime_diff'] = air_reserve.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).seconds 
                                                         * r['reserve_visitors']/3600/24.0, axis=1)


hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].apply(lambda x:x[:10])
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].apply(lambda x:x[:10])
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])
hpg_reserve['reserve_date'] = hpg_reserve['reserve_datetime'].dt.date
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].dt.date
hpg_reserve['reserve_datetime_diff'] = hpg_reserve.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).seconds
                                                         * r['reserve_visitors']/3600/24.0, axis=1)

print ('after')
display(air_reserve.head())
display(hpg_reserve.head())

before


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


after


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,2016-01-01,0.125000
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,2016-01-01,0.000000
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,2016-01-01,0.000000
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,2016-01-01,0.333333
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,2016-01-01,3.958333


,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,reserve_date,reserve_datetime_diff
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,2016-01-01,2016-01-01,0.083333
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,2016-01-01,2016-01-01,0.875000
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,2016-01-01,2016-01-01,0.166667
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,2016-01-01,2016-01-01,1.250000
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,2016-01-01,2016-01-01,7.583333


#### Aggregate reservations

In [13]:
air_reserve_grp = air_reserve.groupby(['air_store_id','visit_date'])['reserve_visitors','reserve_datetime_diff'].\
            sum().reset_index().rename(columns={'reserve_visitors':'air_rvs',
                                               'reserve_datetime_diff':'air_rv_dt_diff'})
hpg_reserve_grp = hpg_reserve.groupby(['hpg_store_id','visit_date'])['reserve_visitors','reserve_datetime_diff'].\
            sum().reset_index().rename(columns={'reserve_visitors':'hpg_rvs',
                                               'reserve_datetime_diff':'hpg_rv_dt_diff'})
air_reserve_grp['mean_air_rv_dt_diff'] = air_reserve_grp.air_rv_dt_diff / air_reserve_grp.air_rvs
hpg_reserve_grp['mean_hpg_rv_dt_diff'] = hpg_reserve_grp.hpg_rv_dt_diff / hpg_reserve_grp.hpg_rvs    
    
display(air_reserve_grp.head())
display(hpg_reserve_grp.head())    

,air_store_id,visit_date,air_rvs,air_rv_dt_diff,mean_air_rv_dt_diff
0,air_00a91d42b08b08d9,2016-10-31,2,0.333333,0.166667
1,air_00a91d42b08b08d9,2016-12-05,9,1.500000,0.166667
2,air_00a91d42b08b08d9,2016-12-14,18,6.750000,0.375000
3,air_00a91d42b08b08d9,2016-12-17,2,0.250000,0.125000
4,air_00a91d42b08b08d9,2016-12-20,4,0.500000,0.125000


,hpg_store_id,visit_date,hpg_rvs,hpg_rv_dt_diff,mean_hpg_rv_dt_diff
0,hpg_001112ef76b9802c,2016-02-26,9,2.250000,0.250000
1,hpg_001112ef76b9802c,2016-03-17,3,0.000000,0.000000
2,hpg_001112ef76b9802c,2016-03-31,5,4.791667,0.958333
3,hpg_001112ef76b9802c,2016-04-05,13,4.333333,0.333333
4,hpg_001112ef76b9802c,2016-04-18,9,1.875000,0.208333


#### Add reservations to full data

In [14]:
full_data = pd.merge(full_data, air_reserve_grp, how='left', on=['air_store_id','visit_date'])
full_data = pd.merge(full_data, hpg_reserve_grp, how='left', on=['hpg_store_id','visit_date'])

display(full_data.query('air_rvs>0 and hpg_rvs>0').head()) 

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,min_lat_hpg_lv2,mean_lon_hpg_lv2,max_lon_hpg_lv2,min_lon_hpg_lv2,air_rvs,air_rv_dt_diff,mean_air_rv_dt_diff,hpg_rvs,hpg_rv_dt_diff,mean_hpg_rv_dt_diff
4669,air_3e93f3c81008696d,NaN,2016-10-27,48,3,2016,10,301,31,43,...,NaN,NaN,NaN,NaN,12.0,7.083333,0.590278,2.0,1.750000,0.875000
4670,air_3e93f3c81008696d,NaN,2016-10-28,25,4,2016,10,302,31,43,...,NaN,NaN,NaN,NaN,4.0,1.833333,0.458333,2.0,1.833333,0.916667
4671,air_3e93f3c81008696d,NaN,2016-10-29,42,5,2016,10,303,31,43,...,NaN,NaN,NaN,NaN,17.0,13.000000,0.764706,28.0,18.416667,0.657738
4673,air_3e93f3c81008696d,NaN,2016-11-01,76,1,2016,11,306,30,44,...,NaN,NaN,NaN,NaN,74.0,66.250000,0.895270,3.0,2.625000,0.875000
4676,air_3e93f3c81008696d,NaN,2016-11-04,5,4,2016,11,309,30,44,...,NaN,NaN,NaN,NaN,11.0,4.625000,0.420455,7.0,2.625000,0.375000


### Date infomation

In [15]:
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])
date_info['dow'] = date_info['visit_date'].dt.dayofweek
date_info['date_len'] = len(date_info)
date_info['date_index'] = date_info.index + 1
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  
date_info['visit_date'] = date_info['visit_date'].dt.date
date_info.head()

,visit_date,day_of_week,holiday_flg,dow,date_len,date_index,weight
0,2016-01-01,Friday,1,4,517,1,2.707368e-14
1,2016-01-02,Saturday,1,5,517,2,8.663577e-13
2,2016-01-03,Sunday,1,6,517,3,6.578904e-12
3,2016-01-04,Monday,0,0,517,4,2.772345e-11
4,2016-01-05,Tuesday,0,1,517,5,8.460525e-11


In [16]:
full_data = pd.merge(full_data, date_info[['visit_date','date_len','date_index','weight', 'holiday_flg']], 
                     how='left', on='visit_date')

## Group variables

In [17]:
num_vars = ['dow', 'year', 'month', 'doy', 'dom', 'woy', 'holiday_flg',
            'date_index', 'date_int', 'air_stores_on_same_addr', 'hpg_stores_on_same_addr',
            'latitude_air', 'longitude_air', 
            'air_stores_on_same_addr', 'air_stores_lv1', 'air_stores_lv2',
            'mean_lat_air_lv1', 'max_lat_air_lv1', 'min_lat_air_lv1',
            'mean_lon_air_lv1', 'max_lon_air_lv1', 'min_lon_air_lv1',
            'mean_lat_air_lv2', 'max_lat_air_lv2', 'min_lat_air_lv2',
            'mean_lon_air_lv2', 'max_lon_air_lv2', 'min_lon_air_lv2',
            'latitude_hpg', 'longitude_hpg', 
            'hpg_stores_on_same_addr', 'hpg_stores_lv1', 'hpg_stores_lv2',
            'mean_lat_hpg_lv1', 'max_lat_hpg_lv1', 'min_lat_hpg_lv1',
            'mean_lon_hpg_lv1', 'max_lon_hpg_lv1', 'min_lon_hpg_lv1',
            'mean_lat_hpg_lv2', 'max_lat_hpg_lv2', 'min_lat_hpg_lv2',
            'mean_lon_hpg_lv2', 'max_lon_hpg_lv2', 'min_lon_hpg_lv2',
            'air_rvs', 'hpg_rvs','air_rv_dt_diff', 'hpg_rv_dt_diff']


cat_vars = ['air_store_id', 'air_genre_name', 'air_area_name', 'air_area_lv1', 'air_area_lv2', 'air_area_lv3',
            'hpg_store_id', 'hpg_genre_name', 'hpg_area_name', 'hpg_area_lv1', 'hpg_area_lv2', 'hpg_area_lv3']

id_var = 'air_store_id'
target_var = 'visitors'

### Missing value imputation

For this week we will leave missing values(Nan) as is and let XGBoost to take care of them automatically.

### Categorical features - label encoding

Label encoding is not really necessary for this competition as all categorical features have already been digitalized. I'm including this just for your reference.


In [18]:
LBL = preprocessing.LabelEncoder()

LE_vars=[]
LE_map=dict()
for cat_var in cat_vars:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var].astype(str))
    LE_vars.append(LE_var)
    LE_map[cat_var]=LBL.classes_
    
print ("Label-encoded feaures: %s" % (LE_vars))

Label Encoding air_store_id
Label Encoding air_genre_name
Label Encoding air_area_name
Label Encoding air_area_lv1
Label Encoding air_area_lv2
Label Encoding air_area_lv3
Label Encoding hpg_store_id
Label Encoding hpg_genre_name
Label Encoding hpg_area_name
Label Encoding hpg_area_lv1
Label Encoding hpg_area_lv2
Label Encoding hpg_area_lv3
Label-encoded feaures: ['air_store_id_le', 'air_genre_name_le', 'air_area_name_le', 'air_area_lv1_le', 'air_area_lv2_le', 'air_area_lv3_le', 'hpg_store_id_le', 'hpg_genre_name_le', 'hpg_area_name_le', 'hpg_area_lv1_le', 'hpg_area_lv2_le', 'hpg_area_lv3_le']


### Categorical features - one hot encoding¶

You don't want to concatenate the converted OHE features with the original dataframe(full_data) becuase it would exponentially enlarge the size of the dataframe. In fact, it's recommended to use scipy.sparse.hstack to concatenate the data which you will see in the following sections.

In [19]:
OHE = preprocessing.OneHotEncoder(sparse=True)
start=time.time()
OHE.fit(full_data[LE_vars])
OHE_sparse=OHE.transform(full_data[LE_vars])
                                   
print ('One-hot-encoding finished in %f seconds' % (time.time()-start))


OHE_vars = [var[:-3] + '_' + str(level).replace(' ','_')\
                for var in cat_vars for level in LE_map[var] ]

print ("OHE_sparse size :" ,OHE_sparse.shape)
print ("One-hot encoded catgorical feature samples : %s" % (OHE_vars[:100]))

One-hot-encoding finished in 1.153341 seconds
OHE_sparse size : (284127, 1377)
One-hot encoded catgorical feature samples : ['air_store_air_00a91d42b08b08d9', 'air_store_air_0164b9927d20bcc3', 'air_store_air_0241aa3964b7f861', 'air_store_air_0328696196e46f18', 'air_store_air_034a3d5b40d5b1b1', 'air_store_air_036d4f1ee7285390', 'air_store_air_0382c794b73b51ad', 'air_store_air_03963426c9312048', 'air_store_air_04341b588bde96cd', 'air_store_air_049f6d5b402a31b2', 'air_store_air_04cae7c1bc9b2a0b', 'air_store_air_0585011fa179bcce', 'air_store_air_05c325d315cc17f5', 'air_store_air_0647f17b4dc041c8', 'air_store_air_064e203265ee5753', 'air_store_air_066f0221b8a4d533', 'air_store_air_06f95ac5c33aca10', 'air_store_air_0728814bd98f7367', 'air_store_air_0768ab3910f7967f', 'air_store_air_07b314d83059c4d2', 'air_store_air_07bb665f9cdfbdfb', 'air_store_air_082908692355165e', 'air_store_air_083ddc520ea47e1e', 'air_store_air_0845d8395f30c6bb', 'air_store_air_084d98859256acf0', 'air_store_air_0867f7beba

## Numeric features

For week 1 and week 2 we will be using XGBoost/LightGBM which typically don't require pre-processing for numeric features so we will skip this part until week 3.

## Feature interactions
### numeric to numeric

In [20]:
full_data['total_rvs'] = full_data['air_rvs'] + full_data['hpg_rvs']
full_data['mean_rvs_air_hpg'] = full_data[['air_rvs','hpg_rvs']].apply(lambda x:np.mean(x), axis=1)
full_data['mean_dt_diff_air_hpg'] = full_data[['air_rv_dt_diff','hpg_rv_dt_diff']].apply(lambda x:np.mean(x), axis=1)

# NEW FEATURES FROM Georgii Vyshnia
full_data['lon_plus_lat_air'] = full_data['longitude_air'] + full_data['latitude_air'] 

full_data['lat_to_mean_lat_air_lv1'] = abs(full_data['latitude_air']-full_data['mean_lat_air_lv1'])
full_data['lat_to_max_lat_air_lv1']  = full_data['latitude_air']-full_data['max_lat_air_lv1']
full_data['lat_to_min_lat_air_lv1']  = full_data['latitude_air']-full_data['min_lat_air_lv1']
full_data['lon_to_mean_lon_air_lv1']  = abs(full_data['longitude_air']-full_data['mean_lon_air_lv1'])
full_data['lon_to_max_lon_air_lv1']  = full_data['longitude_air']-full_data['max_lon_air_lv1']
full_data['lon_to_min_lon_air_lv1']  = full_data['longitude_air']-full_data['min_lon_air_lv1']
full_data['lat_to_mean_lat_air_lv2'] = abs(full_data['latitude_air']-full_data['mean_lat_air_lv2'])
full_data['lat_to_max_lat_air_lv2']  = full_data['latitude_air']-full_data['max_lat_air_lv2']
full_data['lat_to_min_lat_air_lv2']  = full_data['latitude_air']-full_data['min_lat_air_lv2']
full_data['lon_to_mean_lon_air_lv2'] = abs(full_data['longitude_air']-full_data['mean_lon_air_lv2'])
full_data['lon_to_max_lon_air_lv2']  = full_data['longitude_air']-full_data['max_lon_air_lv2']
full_data['lon_to_min_lon_air_lv2']  = full_data['longitude_air']-full_data['min_lon_air_lv2']

full_data['lat_to_mean_lat_hpg_lv1'] = abs(full_data['latitude_hpg']-full_data['mean_lat_hpg_lv1'])
full_data['lat_to_max_lat_hpg_lv1']  = full_data['latitude_hpg']-full_data['max_lat_hpg_lv1']
full_data['lat_to_min_lat_hpg_lv1']  = full_data['latitude_hpg']-full_data['min_lat_hpg_lv1']
full_data['lon_to_mean_lon_hpg_lv1']  = abs(full_data['longitude_hpg']-full_data['mean_lon_hpg_lv1'])
full_data['lon_to_max_lon_hpg_lv1']  = full_data['longitude_hpg']-full_data['max_lon_hpg_lv1']
full_data['lon_to_min_lon_hpg_lv1']  = full_data['longitude_hpg']-full_data['min_lon_hpg_lv1']
full_data['lat_to_mean_lat_hpg_lv2'] = abs(full_data['latitude_hpg']-full_data['mean_lat_hpg_lv2'])
full_data['lat_to_max_lat_hpg_lv2']  = full_data['latitude_hpg']-full_data['max_lat_hpg_lv2']
full_data['lat_to_min_lat_hpg_lv2']  = full_data['latitude_hpg']-full_data['min_lat_hpg_lv2']
full_data['lon_to_mean_lon_hpg_lv2'] = abs(full_data['longitude_hpg']-full_data['mean_lon_hpg_lv2'])
full_data['lon_to_max_lon_hpg_lv2']  = full_data['longitude_hpg']-full_data['max_lon_hpg_lv2']
full_data['lon_to_min_lon_hpg_lv2']  = full_data['longitude_hpg']-full_data['min_lon_hpg_lv2']

num_num_vars = ['total_rvs', 'mean_rvs_air_hpg',
       'mean_dt_diff_air_hpg', 'lon_plus_lat_air',
       'lat_to_mean_lat_air_lv1', 'lat_to_max_lat_air_lv1',
       'lat_to_min_lat_air_lv1', 'lon_to_mean_lon_air_lv1',
       'lon_to_max_lon_air_lv1', 'lon_to_min_lon_air_lv1',
       'lat_to_mean_lat_air_lv2', 'lat_to_max_lat_air_lv2',
       'lat_to_min_lat_air_lv2', 'lon_to_mean_lon_air_lv2',
       'lon_to_max_lon_air_lv2', 'lon_to_min_lon_air_lv2',
       'lat_to_mean_lat_hpg_lv1', 'lat_to_max_lat_hpg_lv1',
       'lat_to_min_lat_hpg_lv1', 'lon_to_mean_lon_hpg_lv1',
       'lon_to_max_lon_hpg_lv1', 'lon_to_min_lon_hpg_lv1',
       'lat_to_mean_lat_hpg_lv2', 'lat_to_max_lat_hpg_lv2',
       'lat_to_min_lat_hpg_lv2', 'lon_to_mean_lon_hpg_lv2',
       'lon_to_max_lon_hpg_lv2', 'lon_to_min_lon_hpg_lv2']

full_data.head()

,air_store_id,id,visit_date,visitors,dow,year,month,doy,dom,woy,...,lat_to_min_lat_hpg_lv1,lon_to_mean_lon_hpg_lv1,lon_to_max_lon_hpg_lv1,lon_to_min_lon_hpg_lv1,lat_to_mean_lat_hpg_lv2,lat_to_max_lat_hpg_lv2,lat_to_min_lat_hpg_lv2,lon_to_mean_lon_hpg_lv2,lon_to_max_lon_hpg_lv2,lon_to_min_lon_hpg_lv2
0,air_ba937bf13d40fb24,NaN,2016-01-13,25,2,2016,1,13,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,NaN,2016-01-14,32,3,2016,1,14,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,NaN,2016-01-15,29,4,2016,1,15,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,NaN,2016-01-16,22,5,2016,1,16,31,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,NaN,2016-01-18,6,0,2016,1,18,31,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### categorical to categorical

In [21]:
full_data['air_area_genre'] = full_data['air_area_name'] + '-' + full_data['air_genre_name'] 
full_data['air_store_dow'] = full_data['air_store_id'] + '-' + full_data['dow'].astype(str)
full_data['air_store_dow_holiday'] = full_data['air_store_id'] + '-' + full_data['dow'].astype(str) + '-' + full_data['holiday_flg'].astype(str)

cat_cat_vars = ['air_area_genre','air_store_dow', 'air_store_dow_holiday']

In [22]:
LBL = preprocessing.LabelEncoder()

cat_cat_LE_vars=[]
cat_cat_LE_map=dict()
for cat_var in cat_cat_vars:
    print ("Label Encoding %s" % (cat_var))
    LE_var=cat_var+'_le'
    full_data[LE_var]=LBL.fit_transform(full_data[cat_var].astype(str))
    cat_cat_LE_vars.append(LE_var)
    cat_cat_LE_map[cat_var]=LBL.classes_
    
print ("Label-encoded feaures: %s" % (cat_cat_LE_vars))

Label Encoding air_area_genre
Label Encoding air_store_dow
Label Encoding air_store_dow_holiday
Label-encoded feaures: ['air_area_genre_le', 'air_store_dow_le', 'air_store_dow_holiday_le']


### Target aggregation

This sometimes works for timeseries problems.

In [23]:
tmp = full_data[:train_size].groupby(['air_store_id','dow','holiday_flg'])['visitors'].\
            agg([np.mean, np.max, np.min, np.median]).\
            reset_index().\
            rename(columns={'mean':'mean_visitors',
                           'amax':'max_visitors',
                           'amin':'min_visitors',
                           'median':'median_visitors'})
            
full_data = pd.merge(full_data, tmp, how='left', on=['air_store_id','dow','holiday_flg'])

tmp = full_data[:train_size].groupby(['air_store_id','dow', 'holiday_flg']).\
            apply(lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )).\
            reset_index().rename(columns={0:'wmean_visitors'})
        
full_data = pd.merge(full_data, tmp, how='left', on=['air_store_id','dow','holiday_flg'])    
target_aggr_vars = ['mean_visitors', 'max_visitors', 'min_visitors', 'median_visitors', 'wmean_visitors']

# Modelling

We all run XGBoost models using a couple of combinations of features as well as with different missing value settings just to see how differently they perform.


## numerical features + label-encoded categorical features
Let's get started with the simplest combination: **numerical features + label-encoded categorical features** without additional transformations.

In [24]:
full_vars = num_vars + LE_vars + num_num_vars + cat_cat_LE_vars + target_aggr_vars
    
train = full_data[:train_size]
y = full_data[:train_size][target_var][:train_size].values
test = full_data[train_size:]
ids = full_data[train_size:][train_size:].id.values

print ('train data size:', train.shape, 'test data size:', test.shape)    

train data size: (252108, 118) test data size: (32019, 118)


In [ ]:
train_x = train[(train['visit_date']<=datetime.datetime.strptime('2017-03-09', '%Y-%m-%d').date()) 
      & (train['visit_date']>datetime.datetime.strptime('2016-04-01', '%Y-%m-%d').date())][full_vars].values
train_y = np.log1p(train[(train['visit_date']<=datetime.datetime.strptime('2017-03-09', '%Y-%m-%d').date()) 
      & (train['visit_date']>datetime.datetime.strptime('2016-04-01', '%Y-%m-%d').date())]['visitors'].values)

val_x = train[(train['visit_date']>datetime.datetime.strptime('2017-03-09', '%Y-%m-%d').date())][full_vars].values
val_y = np.log1p(train[(train['visit_date']>datetime.datetime.strptime('2017-03-09', '%Y-%m-%d').date())]['visitors'].values)
print (train_x.shape, val_x.shape, train_x.shape[0]+ val_x.shape[0])


xgtrain = xgb.DMatrix(train_x, label=train_y)
xgval=xgb.DMatrix(val_x,label=val_y)

watchlist  = [ (xgtrain,'train'),(xgval,'eval')]


best_xgb_params = {'colsample_bytree': 0.7,
 'eta': 0.1,
 'gamma': 1,
 'max_depth': 10,
 'min_child_weight': 3,
 'nthread': 8,
 'objective': 'reg:linear',
 'seed': 1234,
 'subsample': 1}

print (best_xgb_params)

model = xgb.train(best_xgb_params, 
                  xgtrain, 
                  num_boost_round=100000,
                  evals=watchlist,
                  early_stopping_rounds=50,
                  verbose_eval=50)    
best_iteration = model.best_iteration
best_score = model.best_score
print ('best_score: %f, best_iteration: %d' % (best_score, best_iteration))

### Feature importance

In [ ]:
model.feature_names = full_vars
feature_importance = pd.DataFrame.from_dict(model.get_fscore(), orient='index')
feature_importance.columns = ['importance']
feature_importance.importance = feature_importance.importance/ feature_importance.importance.sum()
feature_importance.sort_values(by='importance').head(30).plot(kind='barh',figsize=(4,20))
feature_importance.sort_values(by='importance',ascending=False).head(30)

### Create submission

In [ ]:
train_x = train[full_vars].values
train_y = np.log1p(train['visitors'].values)
model = xgb.train(best_xgb_params, 
                  xgb.DMatrix(train_x, label=train_y), 
                  num_boost_round=best_iteration)    
test['visitors'] = model.predict(xgb.DMatrix(test[full_vars].values))
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub = test[['id','visitors']].copy()
sub[['id', 'visitors']].to_csv('../output/sub_starter.csv', index=False)

# Conclusions

Feature engineering is the key, if not the most important, to the success of a data science projecct including Kaggle competition. It requires a data scientist to have excellent knowledge of Machine Learning algorithms, good sense of business, programming skills and, last but not least, hacker spirits.

In this week's lecture we've learnt how to:

* Preprocess data for
    * Numeric features
    * Categorical features
* Impute missing values
* Select features

# Recommended Kaggle posts:

* [A Very Extensive Recruit Exploratory Analysis](https://www.kaggle.com/captcalculator/a-very-extensive-recruit-exploratory-analysis)
* [https://www.kaggle.com/headsortails/be-my-guest-recruit-restaurant-eda](https://www.kaggle.com/headsortails/be-my-guest-recruit-restaurant-eda)
* [Surprise Me](https://www.kaggle.com/the1owl/surprise-me)
* [Things that make this competition interesting (and fun)](https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting/discussion/45120)


# Additional readings
* [Applied Predictive Modeling - Chapter 3 Data Pre-Processing](http://appliedpredictivemodeling.com/toc/)
* [机器学习特征工程实用技巧大全](https://zhuanlan.zhihu.com/p/26444240)
* [Discover Feature Engineering](http://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/)
* [Selecting good features – Part IV: stability selection, RFE and everything side by side](http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/)


# Assignments
1. Run this notebook and make submissions
2. Experiment whatever feature engineering you could think of and see how they perform